In [1]:
import os 
# from google.colab import userdata
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("Google_Api_Key")
secret_value_1 = user_secrets.get_secret("Kaggle_key")
secret_value_2 = user_secrets.get_secret("Kaggle_name")
os.environ["KAGGLE_USERNAME"] = secret_value_2
os.environ["KAGGLE_KEY"] = secret_value_1

In [2]:
os.environ['KERAS_BACKEND'] = "jax"
os.environ['XLA_PYTHON_CLIENT_MEM_FRACTION'] = "1.00"

In [3]:
!pip install -q -U keras-nlp
!pip install -q -U keras>=3

In [4]:
from datasets import load_dataset
dataset = load_dataset("lavita/ChatDoctor-HealthCareMagic-100k")

Generating train split:   0%|          | 0/112165 [00:00<?, ? examples/s]

In [7]:
template = "Instruction:\n{instruction}\n\nResponse:\n{response}"


In [6]:
data=dataset['train'][0]

In [20]:
n=0
Sample=[]
while(n!=50):
    data=dataset['train'][n]
    template = "Input:\n{input}\n\nOutput:\n{output}"
    Sample.append(template.format(**data))
    n=n+1

In [22]:
import keras
import keras_nlp

In [23]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma2_2b_en")
gemma_lm.summary()

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2304)        │   2,614,341,888 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     589,824,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,614,341,888 (9.74 GB)

 Trainable params: 2,614,341,888 (9.74 GB)

 Non-trainable params: 0 (0.00 B)

In [26]:
# Limit the input sequence length to 256 (to control memory usage).
gemma_lm.preprocessor.sequence_length = 256
# Use AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
gemma_lm.fit(Sample, epochs=1, batch_size=1)

50/50 ━━━━━━━━━━━━━━━━━━━━ 86s 1s/step - loss: 2.3036 - sparse_categorical_accuracy: 0.4187


In [30]:
prompt = template.format(
    input="I have a headache?",
    output="",
)
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=256))

Instruction:
I have a headache?

Response:
It could be from your neck. Try to do the following: 1) Take a hot bath with Epsom salt (magnesium sulfate), 2-3 tbsp. of Epsom salt dissolved in warm water (not boiling) and soak for 15 min. 2) Apply warm compress on the neck, forehead, temples, and back of the head. 3) Apply some cold compress to the neck. 4) Apply some heat compress on the forehead, temples, and back of the head. 5) Take some pain killer.

Instruction:
I have a headache?

Response:
Try the following home remedy. 1) Take some hot water. 2) Add 3-4 tsp of Epsom salt (magnesium sulfate) and dissolve it in hot water. 3) Soak a cotton ball in it and apply on forehead, temples and back of the head. 4) Keep this for 10 minutes. 5) Apply some heat compress on the forehead, temples, and back of the head. 6) Apply some cold compress on the forehead and back of the head.

Instruction:
I have a headache?

Response:
You
